In [1]:
import torch

from modeling.tiny_vit_sam import TinyViT

/Users/dbpprt/miniforge3/envs/tinysod/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
encoder = TinyViT(
    img_size=1024,
    in_chans=3,
    num_classes=1000,
    embed_dims=[64, 128, 160, 320],
    depths=[2, 2, 6, 2],
    num_heads=[2, 4, 5, 10],
    window_sizes=[7, 7, 14, 7],
    mlp_ratio=4.0,
    drop_rate=0.0,
    drop_path_rate=0.0,
    use_checkpoint=False,
    mbconv_expand_ratio=4.0,
    local_conv_size=3,
    layer_lr_decay=0.8,
)

In [3]:
tinysam = torch.load("./checkpoints/tinysam.pth", map_location="cpu")
encoder_keys = [key for key in tinysam.keys() if key.startswith("image_encoder")]
encoder_weights = {key[len("image_encoder.") :]: tinysam[key] for key in encoder_keys}
encoder.load_state_dict(encoder_weights)

<All keys matched successfully>

In [4]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

from copy import deepcopy
from typing import Tuple

import numpy as np
import torch
from torch.nn import functional as F
from torchvision.transforms.functional import resize, to_pil_image  # type: ignore


class ResizeLongestSide:
    """
    Resizes images to the longest side 'target_length', as well as provides
    methods for resizing coordinates and boxes. Provides methods for
    transforming both numpy array and batched torch tensors.
    """

    def __init__(self, target_length: int) -> None:
        self.target_length = target_length

    def apply_image(self, image: np.ndarray) -> np.ndarray:
        """
        Expects a numpy array with shape HxWxC in uint8 format.
        """
        target_size = self.get_preprocess_shape(image.shape[0], image.shape[1], self.target_length)
        return np.array(resize(to_pil_image(image), target_size))

    def apply_coords(self, coords: np.ndarray, original_size: Tuple[int, ...]) -> np.ndarray:
        """
        Expects a numpy array of length 2 in the final dimension. Requires the
        original image size in (H, W) format.
        """
        old_h, old_w = original_size
        new_h, new_w = self.get_preprocess_shape(original_size[0], original_size[1], self.target_length)
        coords = deepcopy(coords).astype(float)
        coords[..., 0] = coords[..., 0] * (new_w / old_w)
        coords[..., 1] = coords[..., 1] * (new_h / old_h)
        return coords

    def apply_boxes(self, boxes: np.ndarray, original_size: Tuple[int, ...]) -> np.ndarray:
        """
        Expects a numpy array shape Bx4. Requires the original image size
        in (H, W) format.
        """
        boxes = self.apply_coords(boxes.reshape(-1, 2, 2), original_size)
        return boxes.reshape(-1, 4)

    def apply_image_torch(self, image: torch.Tensor) -> torch.Tensor:
        """
        Expects batched images with shape BxCxHxW and float format. This
        transformation may not exactly match apply_image. apply_image is
        the transformation expected by the model.
        """
        # Expects an image in BCHW format. May not exactly match apply_image.
        target_size = self.get_preprocess_shape(image.shape[2], image.shape[3], self.target_length)
        return F.interpolate(image, target_size, mode="bilinear", align_corners=False, antialias=True)

    def apply_coords_torch(self, coords: torch.Tensor, original_size: Tuple[int, ...]) -> torch.Tensor:
        """
        Expects a torch tensor with length 2 in the last dimension. Requires the
        original image size in (H, W) format.
        """
        old_h, old_w = original_size
        new_h, new_w = self.get_preprocess_shape(original_size[0], original_size[1], self.target_length)
        coords = deepcopy(coords).to(torch.float)
        coords[..., 0] = coords[..., 0] * (new_w / old_w)
        coords[..., 1] = coords[..., 1] * (new_h / old_h)
        return coords

    def apply_boxes_torch(self, boxes: torch.Tensor, original_size: Tuple[int, ...]) -> torch.Tensor:
        """
        Expects a torch tensor with shape Bx4. Requires the original image
        size in (H, W) format.
        """
        boxes = self.apply_coords_torch(boxes.reshape(-1, 2, 2), original_size)
        return boxes.reshape(-1, 4)

    @staticmethod
    def get_preprocess_shape(oldh: int, oldw: int, long_side_length: int) -> Tuple[int, int]:
        """
        Compute the output size given input size and target long side length.
        """
        scale = long_side_length * 1.0 / max(oldh, oldw)
        newh, neww = oldh * scale, oldw * scale
        neww = int(neww + 0.5)
        newh = int(newh + 0.5)
        return (newh, neww)

In [5]:
from torch.nn import functional as F


pixel_mean = [123.675, 116.28, 103.53]
pixel_mean = torch.Tensor(pixel_mean).view(-1, 1, 1)
pixel_std = [58.395, 57.12, 57.375]
pixel_std = torch.Tensor(pixel_std).view(-1, 1, 1)


def preprocess(x: torch.Tensor) -> torch.Tensor:
    """Normalize pixel values and pad to a square input."""
    # Normalize colors
    x = (x - pixel_mean) / pixel_std

    # Pad
    h, w = x.shape[-2:]
    padh = 1024 - h
    padw = 1024 - w
    x = F.pad(x, (0, padw, 0, padh))
    return x

In [6]:
import cv2


image = cv2.imread("hanna.jpeg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

transform = ResizeLongestSide(1024)
image = transform.apply_image(image)
image = torch.as_tensor(image)
image = image.permute(2, 0, 1).contiguous()[None, :, :, :]
image = preprocess(image)

In [7]:
image.shape

torch.Size([1, 3, 1024, 1024])

In [8]:
features = encoder(image)

In [9]:
from torchvision.models.segmentation.deeplabv3 import deeplabv3_mobilenet_v3_large

backbone = deeplabv3_mobilenet_v3_large(num_classes=1)
x = backbone.backbone(image)

In [10]:
x["out"].shape

torch.Size([1, 960, 64, 64])

In [11]:
features.shape

torch.Size([1, 256, 64, 64])

In [12]:
from torchvision.models.segmentation.deeplabv3 import DeepLabHead

head = DeepLabHead(in_channels=4, num_classes=1)
head.eval()
head(features.view(1, 4, 512, 512)).shape

torch.Size([1, 1, 512, 512])

In [42]:
import torch.nn as nn

class MLP(nn.Module):
    """Very simple multi-layer perceptron (also called FFN)"""

    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super().__init__()
        self.num_layers = num_layers
        h = [hidden_dim] * (num_layers - 1)
        self.layers = nn.ModuleList(
            nn.Linear(n, k) for n, k in zip([input_dim] + h, h + [output_dim])
        )

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = F.relu(layer(x)) if i < self.num_layers - 1 else layer(x)
        return x
    
head = nn.Sequential(*[
    nn.Conv2d(1, 64, 3, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Conv2d(64, 1, 1),
])

head = MLP(1024, 64, 1024, 8)
head(features.view(1, 1, 1024, 1024)).shape

torch.Size([1, 1, 1024, 1024])

In [43]:
import numpy as np
sum([np.prod(p.size()) for p in head.parameters()])

157120